In [1]:
import requests
import pandas as pd

# pd.set_option('display.max_rows', 500)  #lo use para revisar todos los rows del DF
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07")
dfMPasajero2016 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2016["Year"] = 2016 # añado campo año y 2016 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636")
dfMPasajero2017 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2017["Year"] = 2017 # añado campo año y 2017 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5")
dfMViajes2016 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b")
dfMViajes2017 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiLineaAerea/1a8d9e13-ce30-50fc-bf34-6490eb799a75")
dfLAereas = pd.DataFrame.from_dict(response.json())

dfResultPasa = dfMPasajero2016.append(dfMPasajero2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfResultViaje =  dfresultAppend = dfMViajes2016.append(dfMViajes2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017

dfRelList = pd.merge(dfResultPasa, dfResultViaje, left_on = 'ID_Pasajero', right_on='Cve_Cliente', how='inner') #uso Inner para traer todo la informacion en un solo DF

dfNoDupsRelList  = dfRelList.drop_duplicates() #use drop par quitar duplicados

dfNoDupsRelList[['ID_Pasajero', 'Pasajero', 'Edad', 'Cve_LA', 'Viaje', 'Clase', 'Precio','Ruta']]
dfListPreliminar = pd.merge(dfNoDupsRelList, dfLAereas, left_on = 'Cve_LA', right_on = 'Code', how='left')

dfFinal = dfListPreliminar[['Viaje', 'Clase', 'Precio', 'Ruta', 'Edad', 'Linea_Aerea']] #selecciono orde de columnas y las que voy a usar

dfFinaly = dfFinal.rename(columns = {'Viaje': 'Fecha_de_Viaje'}) #Cambio de nombre Viaje a Fecha de Viaje para presentacion
dfFinaly['Fecha_de_Viaje'] = pd.to_datetime(dfFinaly['Fecha_de_Viaje'].astype(str), format='%m/%d/%Y') #cambie el formato de Fecha

dfFinaly['Anio'] = dfFinaly['Fecha_de_Viaje'].dt.year #genere un campo para puro año y facilitar el analisis 
dfFinaly['Mes'] = dfFinaly['Fecha_de_Viaje'].dt.month #genere un campo para puro mes y facilitar el analisis 

#dfFinaly['Anio_Mes'] = (dfFinaly['Fecha_de_Viaje'].dt.year * 100) + dfFinaly['Fecha_de_Viaje'].dt.month #genero un id para combinando año y mes

dfFinaly['Flag_Semestre'] = dfFinaly['Mes'].apply(lambda x: 'Ene-Jun' if x <= 6 else 'Jul-Dic') #usao lambda para crear la logica para sacar el promedio semestral

dfFinaly

dfRes = dfFinaly.groupby(['Anio','Clase', 'Ruta', 'Linea_Aerea', 'Flag_Semestre'])['Precio'].mean() #el uso de mean es para sacar el average o promedio del precio
dfRes = dfRes.to_frame() #La función se utiliza para convertir el objeto de la serie al DataFrame
dfRes = dfRes.rename(columns = {'Precio': 'Precio_Promedio'}) #Cambio el Header Precio a Precio promedio para una mejor vista
dfRes

Precio_Promedio
Anio Clase       Ruta    Linea_Aerea       Flag_Semestre                 
2016 Business    DAL-AMA American Airlines Ene-Jun                   58.0
                                           Jul-Dic                  186.0
                         Avianca           Ene-Jun                  132.0
                         KLM               Ene-Jun                  184.0
                 DAL-ATL Aeromexico        Ene-Jun                  218.0
...                                                                   ...
2017 First Class DAL-TUL Aeromexico        Ene-Jun                   55.0
                                           Jul-Dic                  208.5
                         Avianca           Ene-Jun                   78.0
                                           Jul-Dic                  189.0
                         KLM               Ene-Jun                  242.5

[311 rows x 1 columns]

In [13]:
import requests
import pandas as pd

#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07")
dfMPasajero2016 = pd.DataFrame.from_dict(response.json())
dfMPasajero2016["Year"] = 2016 # añado campo año y 2016 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636")
dfMPasajero2017 = pd.DataFrame.from_dict(response.json())
dfMPasajero2017["Year"] = 2017 # añado campo año y 2017 para saber los datos de que año son

dfresultAppend = dfMPasajero2016.append(dfMPasajero2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfresultAppend

,ID_Pasajero,Pasajero,Edad,Year
0,576,Danielle Thompson,60,2016
1,579,Natalie Cuevas,49,2016
2,683,John Murray,28,2016
3,681,Michael Jacobs,24,2016
4,592,Brian Hunt,40,2016
...,...,...,...,...
95,709,Daniel Rodriguez,49,2017
96,717,Chad Wise,69,2017
97,717,Billy Whitney,37,2017
98,715,Justin Martin,48,2017


In [12]:
import requests
import pandas as pd

#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5")
dfMViajes2016 = pd.DataFrame.from_dict(response.json())


response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b")
dfMViajes2017 = pd.DataFrame.from_dict(response.json())


dfresultAppend = dfMViajes2016.append(dfMViajes2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfresultAppend

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente,Year
0,SW,9/10/2016,Economy,60,DAL-MDW,553,2016
1,AA,6/11/2016,Economy,150,DAL-SLC,554,2016
2,DA,9/21/2016,Economy,68,DAL-AMA,554,2016
3,UA,6/28/2016,Business,160,DAL-SLC,556,2016
4,SW,6/25/2016,Economy,65,DAL-OKC,557,2016
...,...,...,...,...,...,...,...
195,SW,8/16/2017,Economy,229,DAL-TUL,627,2017
196,AM,6/5/2017,First Class,85,DAL-AMA,669,2017
197,AM,6/2/2017,First Class,55,DAL-TUL,693,2017
198,AV,2/9/2017,Business,140,DAL-MDW,637,2017


In [57]:
import requests
import pandas as pd

#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07")
dfMPasajero2016 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2016["Year"] = 2016 # añado campo año y 2016 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636")
dfMPasajero2017 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2017["Year"] = 2017 # añado campo año y 2017 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5")
dfMViajes2016 = pd.DataFrame.from_dict(response.json())


response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b")
dfMViajes2017 = pd.DataFrame.from_dict(response.json())


dfresultPasa = dfMPasajero2016.append(dfMPasajero2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfresultViaje =  dfresultAppend = dfMViajes2016.append(dfMViajes2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017

RelList = pd.merge(dfresultPasa, dfresultViaje, left_on = 'ID_Pasajero', right_on='Cve_Cliente', how='right')

NoDupsRelList  = RelList.drop_duplicates() #use drop par quitar duplicados

NoDupsRelList [['ID_Pasajero', 'Pasajero', 'Edad', 'Cve_LA', 'Viaje', 'Clase', 'Precio','Ruta']] #quito la columna de cve cliente para presentacion

,ID_Pasajero,Pasajero,Edad,Cve_LA,Viaje,Clase,Precio,Ruta
0,553,Nancy Lomonaco,34,SW,9/10/2016,Economy,60,DAL-MDW
1,554,Naresj Patel,25,AA,6/11/2016,Economy,150,DAL-SLC
2,554,Naresj Patel,25,DA,9/21/2016,Economy,68,DAL-AMA
3,556,Nat Gilpin,55,UA,6/28/2016,Business,160,DAL-SLC
4,557,Natalie DeCherney,63,SW,6/25/2016,Economy,65,DAL-OKC
...,...,...,...,...,...,...,...,...
1254,627,Jordan English,52,SW,8/16/2017,Economy,229,DAL-TUL
1257,669,Jason Crane,32,AM,6/5/2017,First Class,85,DAL-AMA
1260,693,Shahid Shariari,67,AM,6/2/2017,First Class,55,DAL-TUL
1266,637,Rob Beeghly,29,AV,2/9/2017,Business,140,DAL-MDW


In [26]:
import requests
import pandas as pd

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07")
dfMPasajero2016 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2016["Year"] = 2016 # añado campo año y 2016 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636")
dfMPasajero2017 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2017["Year"] = 2017 # añado campo año y 2017 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5")
dfMViajes2016 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b")
dfMViajes2017 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiLineaAerea/1a8d9e13-ce30-50fc-bf34-6490eb799a75")
LAereas = pd.DataFrame.from_dict(response.json())


dfresultPasa = dfMPasajero2016.append(dfMPasajero2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfresultViaje =  dfresultAppend = dfMViajes2016.append(dfMViajes2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017

RelList = pd.merge(dfresultPasa, dfresultViaje, left_on = 'ID_Pasajero', right_on='Cve_Cliente', how='right')

NoDupsRelList  = RelList.drop_duplicates() #use drop par quitar duplicados

NoDupsRelList [['ID_Pasajero', 'Pasajero', 'Edad', 'Cve_LA', 'Viaje', 'Clase', 'Precio','Ruta']]

ListPreliminar = pd.merge(NoDupsRelList, LAereas, left_on = 'Cve_LA', right_on = 'Code', how='right')

final = ListPreliminar [['Viaje', 'Clase', 'Precio', 'Ruta', 'Edad', 'Linea_Aerea']] #selecciono orde de columnas y las que voy a usar

Finaly = final.rename(columns={'Viaje': 'Fecha_de_Viaje'}) #Cambio de nombre Viaje a Fecha de Viaje para presentacion

Finaly






,Fecha_de_Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,6/11/2016,Economy,150,DAL-SLC,25,American Airlines
1,6/25/2016,Business,121,DAL-HOU,63,American Airlines
2,6/25/2016,Economy,83,DAL-GRR,63,American Airlines
3,6/25/2016,Economy,141,DAL-TUL,63,American Airlines
4,11/8/2016,Economy,158,DAL-ELP,57,American Airlines
5,5/21/2016,Economy,71,DAL-HOU,48,American Airlines
6,10/26/2016,Economy,55,DAL-AUS,26,American Airlines
7,3/27/2016,Business,157,DAL-AUS,40,American Airlines
8,7/29/2016,Economy,132,DAL-PHX,52,American Airlines
9,2/28/2016,First Class,165,DAL-MDW,40,American Airlines


In [25]:
import requests
import pandas as pd

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#Request a las apis que usare

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07")
dfMPasajero2016 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2016["Year"] = 2016 # añado campo año y 2016 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636")
dfMPasajero2017 = pd.DataFrame.from_dict(response.json())
#dfMPasajero2017["Year"] = 2017 # añado campo año y 2017 para saber los datos de que año son

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5")
dfMViajes2016 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b")
dfMViajes2017 = pd.DataFrame.from_dict(response.json())

response = requests.get("http://analytics.deacero.com/Api/GetApi/ApiLineaAerea/1a8d9e13-ce30-50fc-bf34-6490eb799a75")
dfLAereas = pd.DataFrame.from_dict(response.json())

dfresultPasa = dfMPasajero2016.append(dfMPasajero2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017
dfresultViaje =  dfresultAppend = dfMViajes2016.append(dfMViajes2017) #Use Appen para poner los datos como  lista, usando merge los ponia los datos 2016 continuos al 2017

#print(dfresultPasa.head())
#print(dfresultViaje.head())

dfRelList = pd.merge(dfresultPasa, dfresultViaje, left_on = 'ID_Pasajero', right_on='Cve_Cliente', how='inner')

dfNoDupsRelList  = dfRelList.drop_duplicates() #use drop par quitar duplicados

dfNoDupsRelList[['ID_Pasajero', 'Pasajero', 'Edad', 'Cve_LA', 'Viaje', 'Clase', 'Precio','Ruta']]
dfListPreliminar = pd.merge(dfNoDupsRelList, dfLAereas, left_on = 'Cve_LA', right_on = 'Code', how='left')

#print(dfListPreliminar.count())
#print(LAereas)

dfFinal = dfListPreliminar[['Viaje', 'Clase', 'Precio', 'Ruta', 'Edad', 'Linea_Aerea']] #selecciono orde de columnas y las que voy a usar

dfFinaly = dfFinal.rename(columns = {'Viaje': 'Fecha_de_Viaje'}) #Cambio de nombre Viaje a Fecha de Viaje para presentacion
dfFinaly['Fecha_de_Viaje'] = pd.to_datetime(dfFinaly['Fecha_de_Viaje'].astype(str), format='%m/%d/%Y')

dfFinaly['Anio'] = dfFinaly['Fecha_de_Viaje'].dt.year
dfFinaly['Mes'] = dfFinaly['Fecha_de_Viaje'].dt.month

dfFinaly['Anio_Mes'] = (dfFinaly['Fecha_de_Viaje'].dt.year * 100) + dfFinaly['Fecha_de_Viaje'].dt.month

dfFinaly['Flag_Semestre'] = dfFinaly['Mes'].apply(lambda x: 'Ene-Jun' if x <= 6 else 'Jul-Dic')

dfFinaly

dfRes = dfFinaly.groupby(['Anio','Clase', 'Ruta', 'Linea_Aerea', 'Flag_Semestre'])['Precio'].mean()
dfRes = dfRes.to_frame()
dfRes = dfRes.rename(columns = {'Precio': 'Precio_Promedio'})
dfRes

Precio_Promedio
Anio Clase       Ruta    Linea_Aerea       Flag_Semestre                 
2016 Business    DAL-AMA American Airlines Ene-Jun              58.000000
                                           Jul-Dic             186.000000
                         Avianca           Ene-Jun             132.000000
                         KLM               Ene-Jun             184.000000
                 DAL-ATL Aeromexico        Ene-Jun             218.000000
                         American Airlines Ene-Jun             102.000000
                                           Jul-Dic             135.000000
                         Avianca           Jul-Dic             107.000000
                         KLM               Jul-Dic             227.000000
                         Southwest         Jul-Dic             185.000000
                 DAL-AUS Aeromexico        Ene-Jun             250.000000
                         American Airlines Ene-Jun             186.500000
                         KLM               Ene-Jun             258.000000
                                           Jul-Dic             191.000000
                 DAL-ELP Aeromexico        Jul-Dic             176.000000
                         American Airlines Jul-Dic             114.000000
                         Avianca           Ene-Jun             173.000000
                                           Jul-Dic             212.000000
                         KLM               Ene-Jun             153.000000
                                           Jul-Dic             162.000000
                 DAL-GRR Aeromexico        Ene-Jun             269.000000
                         American Airlines Ene-Jun             221.000000
                                           Jul-Dic             180.000000
                         Avianca           Ene-Jun             149.000000
                         Southwest         Ene-Jun             266.000000
                 DAL-HOU Aeromexico        Ene-Jun              76.000000
                                           Jul-Dic             127.500000
                         American Airlines Ene-Jun             121.000000
                         Avianca           Ene-Jun             172.000000
                                           Jul-Dic              99.000000
                         KLM               Ene-Jun              62.000000
                         Southwest         Ene-Jun              71.000000
                 DAL-LGA American Airlines Ene-Jun             215.000000
                                           Jul-Dic             216.000000
                         Avianca           Ene-Jun             191.000000
                         Southwest         Ene-Jun              97.000000
                                           Jul-Dic             100.000000
                 DAL-MDW Aeromexico        Jul-Dic             239.000000
                         American Airlines Jul-Dic              77.000000
                         Southwest         Jul-Dic             171.000000
                 DAL-OKC Aeromexico        Jul-Dic             142.000000
                         American Airlines Ene-Jun             181.000000
                                           Jul-Dic             157.000000
                         Avianca           Jul-Dic             132.000000
                         Southwest         Ene-Jun              89.000000
                                           Jul-Dic              59.000000
                 DAL-PHX Avianca           Ene-Jun             236.000000
                         Southwest         Ene-Jun             202.000000
                                           Jul-Dic              81.000000
                 DAL-SLC Avianca           Ene-Jun             117.000000
                         KLM               Ene-Jun              75.000000
                                           Jul-Dic             209.000000
                         Southwest         Ene-Jun            